In [1]:
%matplotlib inline

import numpy as np
from pprint import pprint

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import torchvision
from torchvision import models, datasets, transforms

torch.manual_seed(50)

print(torch.__version__, torchvision.__version__)

2.0.0+cu118 0.15.1+cu118


In [2]:
# 加载CIFAR-10数据集，如果本地没有则自动下载
dst = datasets.LFWPeople("/home/raoxy/data",download=True)

Files already downloaded and verified


In [3]:
# 定义一个转换序列，包括缩放、裁剪、转换为张量等操作
tp = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor()
])

# 定义一个函数，用于将张量转换为PIL图像
tt = transforms.ToPILImage()

# 检测是否有可用的GPU，如果有则使用GPU，否则使用CPU
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
print("Running on %s" % device)

Running on cuda


In [4]:
# 定义一个函数，用于将标签转换为one-hot编码的张量
def label_to_onehot(target, num_classes=10):
    # 在第一个维度上增加一个维度，使得target的形状为(N, 1)
    target = torch.unsqueeze(target, 1)
    # 创建一个全零的张量，形状为(N, num_classes)，设备与target相同
    onehot_target = torch.zeros(target.size(0), num_classes, device=target.device)
    # 在第二个维度上根据target的值将onehot_target的对应位置设为1
    onehot_target.scatter_(1, target, 1)
    # 返回onehot_target
    return onehot_target

# 定义一个函数，用于计算one-hot编码的标签和预测值之间的交叉熵损失
def cross_entropy_for_onehot(pred, target):
    # 对预测值进行log_softmax操作，然后与目标值相乘，再求和，最后求平均
    return torch.mean(torch.sum(- target * F.log_softmax(pred, dim=-1), 1))

# LeNet

In [5]:
# 定义一个类，继承自nn.Module，表示LeNet模型
class LeNet(nn.Module):
    def __init__(self,out_features):
        super(LeNet, self).__init__()
        self.body = nn.Sequential(
            nn.Conv2d(3, 12, kernel_size=5, stride=2, padding=2),
            nn.Sigmoid(),
            nn.Conv2d(12, 12, kernel_size=5, stride=2, padding=2),
            nn.Sigmoid(),
            nn.Conv2d(12, 12, kernel_size=5, stride=1, padding=2),
            nn.Sigmoid(),
            nn.Conv2d(12, 12, kernel_size=5, stride=1, padding=2),
            nn.Sigmoid()
        )
        self.fc = nn.Sequential(
            nn.Linear(768, out_features, bias=True) # 修改输出特征数为 5749
        )

    def forward(self, x):
        x = self.body(x)
        x = x.view(-1, 768)
        x = self.fc(x)
        return x
    
# 创建一个LeNet模型的实例，并将其移动到设备上，可以是CPU或GPU
net = LeNet(5749).to(device)
net

LeNet(
  (body): Sequential(
    (0): Conv2d(3, 12, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): Sigmoid()
    (2): Conv2d(12, 12, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (3): Sigmoid()
    (4): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): Sigmoid()
    (6): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): Sigmoid()
  )
  (fc): Sequential(
    (0): Linear(in_features=768, out_features=5749, bias=True)
  )
)

In [6]:
# 定义一个函数，用于初始化模型的权重和偏置
def weights_init(m):
    # 如果模型有权重属性，就将其均匀分布在[-0.5, 0.5]之间
    if hasattr(m, "weight"):
        m.weight.data.uniform_(-0.5, 0.5)
    # 如果模型有偏置属性，也将其均匀分布在[-0.5, 0.5]之间
    if hasattr(m, "bias"):
        m.bias.data.uniform_(-0.5, 0.5)
        
    
# 调用weights_init函数，对模型的参数进行初始化
net.apply(weights_init)
# 定义损失函数为one-hot编码的交叉熵损失
criterion = cross_entropy_for_onehot

# 图像索引

In [7]:
# KSVD
def K_Soteria(img_index):
    global History
    from PIL import Image
    import matplotlib.pyplot as plt

    gt_data = dst[img_index][0] # 从数据集中获取图像，并使用之前定义的转换序列处理图像
    img_array = np.array(gt_data)
    
    # 分离三个颜色通道
    R = img_array[:, :, 0]
    G = img_array[:, :, 1]
    B = img_array[:, :, 2]

    # 定义一个函数，用SVD对单通道矩阵进行压缩
    def compress_channel(data, k):
        from  ksvd import ApproximateKSVD
        ksvd = ApproximateKSVD(n_components=k)
        data = data.astype(np.float32)  # 将 data 转换为 float32 类型
        output_data =np.zeros_like(data)
        output_D = ksvd.fit(data).components_
        output_L = ksvd.transform(data)
        output_data =output_L.dot(output_D)
        return output_data

    k=33
    # 对每个颜色通道进行压缩
    R_k = compress_channel(R, k)
    G_k = compress_channel(G, k)
    B_k = compress_channel(B, k)
    
    # 将压缩后的三个颜色通道合并为一个三维数组
    img_array_k = np.stack((R_k, G_k, B_k), axis=2)
    # 将压缩后的数组转换为图片
    img_k = Image.fromarray(img_array_k.astype('uint8'))
    # 从数据集中获取图像，并使用之前定义的转换序列处理图像
    gt_data = tp(img_k).to(device)

    # 将图像的形状调整为(1, 3, 32, 32)，表示批量大小为1，通道数为3，高度和宽度为32
    gt_data = gt_data.view(1, *gt_data.size())
    # 从数据集中获取图像的标签，并转换为长整型张量
    gt_label = torch.Tensor([dst[img_index][1]]).long().to(device)
    # 将标签的形状调整为(1,)，表示批量大小为1
    gt_label = gt_label.view(1, )
    # 将标签转换为one-hot编码的张量，形状为(1, 10)，表示批量大小为1，类别数为10
    gt_onehot_label = label_to_onehot(gt_label, num_classes=10)
    # 设置图像的梯度属性为True，表示可以对图像进行梯度计算
    gt_data.requires_grad = True

    # 计算输出和特征向量相对于输入的导数的范数与特征向量的范数的比值，即||dr/dX||/||r||
    out,feature_fc1_graph = net(gt_data) # 通过网络得到输出和特征向量
    deviation_f1_target = torch.zeros_like(feature_fc1_graph) # 创建一个全零的张量，用于存储目标梯度
    deviation_f1_x_norm = torch.zeros_like(feature_fc1_graph) # 创建一个全零的张量，用于存储导数的范数
    for f in range(deviation_f1_x_norm.size(1)): # 对于每个特征向量的维度
        deviation_f1_target[:,f] = 1 # 将目标梯度的对应位置设为1
        feature_fc1_graph.backward(deviation_f1_target, retain_graph=True) # 对特征向量进行反向传播，计算梯度
        deviation_f1_x = gt_data.grad.data # 获取输入的梯度
        deviation_f1_x_norm[:,f] = torch.norm(deviation_f1_x.view(deviation_f1_x.size(0), -1), dim=1)/(feature_fc1_graph.data[:,f]) # 计算梯度的范数与特征向量的比值
        net.zero_grad() # 清零网络的梯度
        gt_data.grad.data.zero_() # 清零输入的梯度
        deviation_f1_target[:,f] = 0 # 将目标梯度的对应位置设为0

    # 根据最小的||dr_i/dX||/||r_i||来剪枝特征向量
    deviation_f1_x_norm_sum = deviation_f1_x_norm.sum(axis=0) # 对每个维度求和
    thresh = np.percentile(deviation_f1_x_norm_sum.flatten().cpu().numpy(), 1) # 根据百分位数确定阈值
    mask = np.where(abs(deviation_f1_x_norm_sum.cpu()) < thresh, 0, 1).astype(np.float32) # 根据阈值生成掩码，小于阈值的为0，大于阈值的为1
    y = criterion(out, gt_onehot_label) # 计算输出和标签之间的损失
    dy_dx = torch.autograd.grad(y, net.parameters()) # 计算损失对网络参数的梯度

    # 与其他客户端共享梯度
    original_dy_dx = list((_.detach().clone() for _ in dy_dx)) # 复制梯度
    original_dy_dx[-2] = original_dy_dx[-2] * torch.Tensor(mask).to(device)# 将梯度乘以掩码，实现剪枝

    # 生成一些随机的数据和标签，形状与真实的数据和标签相同
    dummy_data_init = torch.randn(gt_data.size()) # 使用torch.randn()函数[^1^][1]生成一个服从标准正态分布的张量
    dummy_label_init = torch.randn(gt_onehot_label.size()) # 同上

    # 将随机的数据和标签转换为张量，并移动到设备上，可以是CPU或GPU
    dummy_data = torch.Tensor(dummy_data_init).to(device).requires_grad_(True) # 使用torch.Tensor()函数将numpy数组转换为张量，并设置requires_grad属性为True，表示可以对数据进行梯度计算
    dummy_label = torch.Tensor(dummy_label_init).to(device).requires_grad_(True) # 同上

    # 定义一个LBFGS优化器，将虚拟的数据和标签作为需要优化的参数
    optimizer = torch.optim.LBFGS([dummy_data, dummy_label] )
    # 注释掉了另一个优化器，SGD，可能是为了比较效果
    #optimizer = torch.optim.SGD([dummy_data, dummy_label], lr=0.1, momentum=0.9 )

    # 创建一个空列表，用于存储优化过程中的虚拟数据
    history = []
    # 定义一个变量，用于存储虚拟数据和真实数据之间的最小均方误差
    MSE_min = 100
    # 进行300次迭代优化
    for iters in range(300):
        # 定义一个闭包函数，用于计算虚拟数据和标签的梯度差
        def closure():
            # 清零优化器的梯度
            optimizer.zero_grad()
            # 通过网络得到虚拟数据的预测和特征向量
            #out, [feature_fc1_graph, feature_fc2_graph, feature_fc3_graph] = net(gt_data)
            pred, f1 = net(dummy_data) 
            # 对虚拟标签进行softmax操作，得到one-hot编码的张量
            dummy_onehot_label = F.softmax(dummy_label, dim=-1)
            # 计算预测和虚拟标签之间的损失，使用之前定义的交叉熵损失函数
            dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
            # 计算损失对网络参数的梯度，创建计算图
            dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)

            # 初始化梯度差和梯度数量为0
            grad_diff = 0
            grad_count = 0

            i = 0
            for gx, gy in zip(dummy_dy_dx, original_dy_dx): # TODO: fix the variablas here
                # 只计算前100个参数的梯度差，可能是为了节省计算资源
                if i <=100:
                    grad_diff += ((gx - gy) ** 2).sum()
                    grad_count += gx.nelement()
                i += 1
            # 计算梯度差的平均值，乘以一个系数，可能是为了调整梯度的大小
            #grad_diff = grad_diff / grad_count * 1000

            # 计算特征向量和真实数据的差的平方和，可能是另一种计算梯度差的方法
            #grad_diff = ((feature_fc1_graph - f1) ** 2).sum()
            # 对梯度差进行反向传播，计算虚拟数据和标签的梯度
            grad_diff.backward()

            # 返回梯度差
            return grad_diff

        # 调用优化器的step()方法，使用闭包函数进行一步优化
        optimizer.step(closure)
        # 如果当前的均方误差小于之前的最小值，就更新最小值
        if MSE_min > (gt_data[0] - dummy_data[0]).pow(2).mean().item():
            MSE_min = (gt_data[0] - dummy_data[0]).pow(2).mean().item()
        # 每隔10次迭代，打印当前的迭代次数，梯度差，均方误差和最小均方误差
        if iters % 10 == 0: 
            current_loss = closure()
            # print("{}, loss: {}, MSE: {}, MSE_min: {}".format(iters, current_loss.item(), (gt_data[0] - dummy_data[0]).pow(2).mean().item(), MSE_min))
        # 将当前的虚拟数据添加到历史列表中
        history.append(tt(dummy_data[0].cpu()))
    Print(History)
    return history

In [8]:
def Print(History):
    plt.figure(figsize=(12, 8))
    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    for j in range(min(50,len(History))):
        plt.subplot(5, 10, j+1)
        plt.imshow(tt(tp(History[j])))
        plt.title("title={}".format(j))
        plt.axis('off')
    plt.show()

In [9]:
History=[]

for i in range(50):
    History.append(K_Soteria(i)[-1])
    print("#"*50+"["+ str(i)+"]"+"#"*50)

../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [0,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.


RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR

In [ ]:
# 画图
plt.figure(figsize=(12, 6))
plt.subplots_adjust(wspace=0.05, hspace=0.05)

for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.imshow(dst[i][0])
#     plt.title((dir_label[dst[i][1]]))
    plt.axis('off')

In [ ]:
# 画图
plt.figure(figsize=(12, 8))
plt.subplots_adjust(wspace=0.05, hspace=0.05)

for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.title("{}".format(i))
    plt.imshow(History[i])
    plt.axis('off')

In [ ]:
for i in range(50):
    plt.axis('off')
    plt.imshow(History[i])
    plt.savefig('image_dlg_f_{}.png'.format(i), bbox_inches='tight')

In [ ]:
for i in range(50):
    plt.axis('off')
    plt.imshow(dst[i][0])
    plt.savefig('image_r_f_{}.png'.format(i), bbox_inches='tight')

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr
psnr(np.array(dst[img_index][0]), np.array(history[-1]))

# 导入必要的库
from skimage.metrics import structural_similarity as ssim # 使用skimage库中的SSIM函数
import numpy as np # 使用numpy库进行数组操作

# 定义SSIM计算函数
def compare_ssim(img1, img2, maxvalue=255):
    # 将图像转换为浮点数类型
    img1 = np.array(img1).astype(np.float64)
    img2 = np.array(img2).astype(np.float64)
    # 调用skimage库中的SSIM函数，指定数据范围为maxvalue
    return ssim(img1, img2, data_range=maxvalue,win_size=3)

In [ ]:
compare_ssim(dst[img_index][0], history[-1], maxvalue=255)